In [1]:
import CalculatedFieldSubroutines as cfs

import pandas as pd

import numpy as np

from pandasgui import show

#

import warnings

In [2]:
warnings.filterwarnings( 'ignore' )

In [3]:
def list_whitelisted_gmIDs_with_traffic_data():

    whitelisted_gmIDs_list = cfs.list_whitelisted_gmIDs()

    whitelisted_gmIDs_without_traffic_data_list = [ '69ab88ec-dc17-11ee-a158-97f8443fd730', 
                                                    '88a68dd8-eef9-11ee-9385-ef789ffde1d3', 
                                                    'fc211bb2-efca-11ee-b966-fb353e7798cd' ]

    whitelisted_gmIDs_with_traffic_data = list( set( whitelisted_gmIDs_list ) - set( whitelisted_gmIDs_without_traffic_data_list ) )

    return whitelisted_gmIDs_with_traffic_data

In [4]:
def retrieve_gmID_preprocessed_moving_data( gmID, window_seconds ):

    path = f'{ cfs.origin_dir() }/Preprocessed_Moving_Data/{ window_seconds }sec_Window/{ gmID }/{ gmID }_{ window_seconds }sec.csv'

    gmID_preprocessed_moving_data_df = pd.read_csv( path )

    return gmID_preprocessed_moving_data_df

In [5]:
def DisengagementID( df, expanded = False ):

    gmID = df[ 'groupMetadataID' ][ 0 ]
    
    #

    disengagment_col = np.array( df[ 'BinaryDisengagement' ] )

    disengagement_indexes = np.array( np.where( disengagment_col == 1 )[ 0 ] )

    #

    DisengagementID_col = [ 'NAD' for i in disengagment_col ]

    #

    if ( expanded == False ):

        string = ''

        #
        
        counter = 0

        for index in disengagement_indexes:

            DisengagementID_col[ int( index ) ] = f'{ gmID }_D{ counter }'

            counter = counter + 1

    elif ( expanded == True ):

        string = 'Expanded'

        #

        expanded_disengagment_col = np.array( df[ 'BinaryDisengagementExpanded' ] )

        expanded_disengagement_indexes = np.where( expanded_disengagment_col == 1 )[ 0 ]

        #

        counter = 0

        for expanded_disengagement_index in expanded_disengagement_indexes:

            temp_array = disengagement_indexes - expanded_disengagement_index

            #

            temp_array2 = disengagement_indexes[ np.where( temp_array >= 0 ) ]

            #

            disengagement_index = int( temp_array2[ 0 ] )

            #

            disengagement_num = int( np.where( disengagement_indexes == disengagement_index )[ 0 ][ 0 ] )

            #

            expanded_disengagement_num = disengagement_index - expanded_disengagement_index

            #

            DisengagementID_col[ expanded_disengagement_index ] = f'{ gmID }_D{ disengagement_num }_ED{ expanded_disengagement_num }'

    #

    df[ f'Disengagement{ string }ID' ] = DisengagementID_col

In [6]:
def RelaxedBinaryDisengagement( df, moving_colname, window_radius, expanded = False ):

    moving_col = np.array( df[ moving_colname ] )

    #

    if ( expanded == False ):

        string = ''

        disengagment_col = np.array( df[ 'BinaryDisengagement' ] )

    elif ( expanded == True ):

        string = 'Expanded'

        disengagment_col = np.array( df[ 'BinaryDisengagementExpanded' ] )

    #

    RelaxedBinaryDisengagement_col = []

    for value in moving_col:

        min_value = value - window_radius

        max_value = value + window_radius

        #

        window_indexes = np.where( ( moving_col >= min_value ) & ( moving_col <= max_value ) )

        #

        disengagment_window_set = set( disengagment_col[ window_indexes ] )

        if ( len( disengagment_window_set ) == 1 ):

            if ( 0 in disengagment_window_set ):

                RelaxedBinaryDisengagement_col.append( 0 )

            else:

                RelaxedBinaryDisengagement_col.append( 1 )

        else:

            RelaxedBinaryDisengagement_col.append( 10 )

    #

    df[ f'RelaxedBinaryDisengagement{ string }' ] = RelaxedBinaryDisengagement_col

In [7]:
def confusion_matrix_values( prediction_col, true_col ):

    true_positive_num = 0

    true_negative_num = 0

    false_positive_num = 0

    false_negative_num = 0

    for predicted_value, true_value in zip( prediction_col, true_col ):

        if ( ( predicted_value == 0 ) and ( true_value == 0 ) ):

            true_negative_num = true_negative_num + 1

        #

        elif ( ( predicted_value == 1 ) and ( true_value == 1 ) ):

            true_positive_num = true_positive_num + 1

        #

        elif ( ( predicted_value == 0 ) and ( true_value == 1 ) ):

            false_negative_num = false_negative_num + 1

        #

        elif ( ( predicted_value == 1 ) and ( true_value == 0 ) ):

            false_positive_num = false_positive_num + 1

        #

        else:

            raise Exception( 'Non 0 or 1 value present in either column' )

    #

    return true_positive_num, true_negative_num, false_positive_num, false_negative_num

In [8]:
gmIDs = list_whitelisted_gmIDs_with_traffic_data()

In [9]:
x = cfs.retrieve_gmID_preprocessed_moving_data( gmID = gmIDs[ 0 ], window_seconds = 10 )

cfs.BinaryDisengagementExpanded( x, moving_colname = 'time', window = 1e9 )

cfs.DisengagementID( x, expanded = False )

cfs.DisengagementID( x, expanded = True )

In [10]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51210 entries, 0 to 51209
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Ind                          51210 non-null  int64  
 1   groupMetadataID              51210 non-null  object 
 2   time                         51210 non-null  int64  
 3   NormalizedTime               51210 non-null  float64
 4   BinaryDrivingMode            51210 non-null  int64  
 5   BinaryDisengagement          51210 non-null  int64  
 6   latitude                     51210 non-null  float64
 7   longitude                    51210 non-null  float64
 8   ProgressAlongRoute           51210 non-null  float64
 9   solStatus                    51210 non-null  object 
 10  solType                      51210 non-null  object 
 11  extendedSolutionStatus       51210 non-null  int64  
 12  numSatsInSolution            51210 non-null  int64  
 13  speedMps        

In [11]:
show( x[ [ 'BinaryDisengagement', 'DisengagementID', 'BinaryDisengagementExpanded', 'DisengagementExpandedID' ] ] )

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [12]:
cfs.confusion_matrix_values( prediction_col = x[ 'BinaryDisengagement' ], true_col = x[ 'BinaryDisengagementExpanded' ] )

(15, 50753, 0, 442)